# **DESAFÍO Nº5: INFERENCIA E HIPÓTESIS**
### **Módulo 4: Análisis Estadístico con Python**
#### *Carrera Data Science - G-93 - Desafio Latam*
#### Estudiante: Felipe Leal Arancibia

En este desafío validaremos nuestros conocimientos de estadística inferencial. Para lograrlo, realizarás inferencias y pruebas de hipótesis a partir de la base de datos
earnings.csv.

Lee todo el documento antes de comenzar el desarrollo individual, para asegurarte de tener el máximo de puntaje y enfocar bien los esfuerzos.

Tiempo asociado: 2 horas cronológicas

## Descripción

#### Podemos registrar muchas características de una persona, y buscar ver si algunas influyen o no en otras. Realiza las siguientes actividades para verificarlo.

In [1]:
# Primero importar todas las potenciales librería sa utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


#### 1. Considerando las variables earn, height y age, vamos a suponer que nuestro dataset fuera una población completa. (Prepara los datos adecuadamente).

In [4]:

# Cargamos el dataset
df_earnings = pd.read_csv('earnings.csv')

df_earnings.shape

(1816, 15)

In [5]:
df_earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   height            1816 non-null   int64  
 1   weight            1789 non-null   float64
 2   male              1816 non-null   int64  
 3   earn              1816 non-null   float64
 4   earnk             1816 non-null   float64
 5   ethnicity         1816 non-null   object 
 6   education         1814 non-null   float64
 7   mother_education  1572 non-null   float64
 8   father_education  1521 non-null   float64
 9   walk              1816 non-null   int64  
 10  exercise          1816 non-null   int64  
 11  smokenow          1815 non-null   float64
 12  tense             1815 non-null   float64
 13  angry             1815 non-null   float64
 14  age               1816 non-null   int64  
dtypes: float64(9), int64(5), object(1)
memory usage: 212.9+ KB


In [6]:
df_earnings.tail(3)

,height,weight,male,earn,earnk,ethnicity,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
1813,72,194.0,1,60000.0,60.0,White,12.0,12.0,12.0,2,1,2.0,0.0,0.0,50
1814,63,155.0,0,15000.0,15.0,Other,14.0,14.0,14.0,6,1,2.0,2.0,2.0,69
1815,68,150.0,1,6000.0,6.0,White,12.0,12.0,12.0,1,6,1.0,2.0,2.0,27


In [7]:
# Evalúo ahora cuántos valores nulos hay, y los ordeno

df_earnings.apply(lambda x: sum(x.isnull()), axis=0).sort_values(ascending=False)



father_education    295
mother_education    244
weight               27
education             2
tense                 1
smokenow              1
angry                 1
ethnicity             0
earnk                 0
earn                  0
male                  0
height                0
walk                  0
exercise              0
age                   0
dtype: int64

In [8]:
# función para contar y listar los valores nulos de una columna
def listar_nulos(df, columna):

    datos_nulos = df[columna].isnull()
    cantidad_nulos = datos_nulos.sum()
    porcentaje_nulos = round(cantidad_nulos / len(df[columna]) * 100, 2)

    return f"Nombre columna: '{columna}'; Cantidad de nulos: {cantidad_nulos}; Porcentaje de nulos en columna: {porcentaje_nulos}%"

In [9]:
listar_nulos(df_earnings, 'father_education')

"Nombre columna: 'father_education'; Cantidad de nulos: 295; Porcentaje de nulos en columna: 16.24%"

Esa columna 'father_education' es la que más valores nulos tiene, y representa a menos de 1/5 de la muestra. Lo dejaré con valores nan por ahora, hasta que se necesite relizar acción distinta.

In [12]:
round(df_earnings.describe(),1)

,height,weight,male,earn,earnk,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
count,1816.0,1789.0,1816.0,1816.0,1816.0,1814.0,1572.0,1521.0,1816.0,1816.0,1815.0,1815.0,1815.0,1816.0
mean,66.6,156.3,0.4,21147.3,21.1,13.2,13.6,13.7,5.3,3.0,1.7,1.4,1.4,42.9
std,3.8,34.6,0.5,22531.8,22.5,2.6,3.2,3.3,2.6,2.3,0.4,2.2,2.2,17.2
min,57.0,80.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,1.0,1.0,0.0,0.0,18.0
25%,64.0,130.0,0.0,6000.0,6.0,12.0,12.0,12.0,3.0,1.0,1.0,0.0,0.0,29.0
50%,66.0,150.0,0.0,16000.0,16.0,12.0,13.0,13.0,6.0,2.0,2.0,0.0,0.0,39.0
75%,69.2,180.0,1.0,27000.0,27.0,15.0,16.0,16.0,8.0,5.0,2.0,2.0,2.0,56.0
max,82.0,342.0,1.0,400000.0,400.0,18.0,99.0,99.0,8.0,7.0,2.0,7.0,7.0,91.0


Allí se puede ver el cuadro síntesis de nuestra  "polación completa", a tener en cuenta en pasos siguientes.

##### a. Realiza 5 pruebas con muestras de tamaño n = 25, en cada caso, con niveles de significancia diferentes pero inferiores a 0,1, para confirmar o rechazar las siguientes hipótesis considerando las alternativas. Crea para ello una fórmula que reciba los parámetros adecuados y responda "con nivel de confianza del ...%, se acepta la hipótesis nula/alternativa".

earn                        height                  age
H0: x = 20000        |       H0: x = 55       |       H0: x = 44
H1: x ≠ 20000        |       H1: x ≠ 55       |       H1: x ≠ 44

H0: x = 35000        |       H0: x = 68        |      H0: x = 56
H1: x ≠ 35000        |       H1: x ≠ 68        |      H1: x ≠ 56

H0: x = 25000        |       H0: x = 63        |      H0: x = 51
H1: x > 25000        |       H1: x > 63        |      H1: x > 51

H0: x = 32000        |       H0: x = 70        |      H0: x = 62
H1: x < 32000        |       H1: x < 70        |      H1: x < 62

In [22]:
def prueba_hipotesis(data, variable, h0, h1, alpha):
    # Tomar una muestra de tamaño n=25
    muestra = data[variable].sample(n=25, random_state=1)
    
    # Calcular la media y el estadístico t
    media_muestra = muestra.mean()
    estadistico_t = (media_muestra - h0) / (muestra.std(ddof=1) / np.sqrt(len(muestra)))
    
    # Calcular el valor crítico
    valor_critico = stats.t.ppf(1 - alpha/2, df=len(muestra)-1)
    
    # Determinar si se rechaza o no la hipótesis nula
    if abs(estadistico_t) > valor_critico:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se rechaza la hipótesis nula"
    else:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se acepta la hipótesis nula"
    
    return resultado

# Definir las hipótesis y niveles de significancia
hipotesis = [
    ('earn', 20000, 'x ≠ 20000', 0.05),
    ('height', 55, 'x ≠ 55', 0.05),
    ('age', 44, 'x ≠ 44', 0.05),
    ('earn', 35000, 'x ≠ 35000', 0.01),
    ('height', 68, 'x ≠ 68', 0.01)
]

# Aplicar la función a cada hipótesis
resultados = [prueba_hipotesis(df_earnings, var, h0, h1, alpha) for var, h0, h1, alpha in hipotesis]

# Mostrar los resultados
for i, resultado in enumerate(resultados):
    print(f"Hipótesis {i+1}: {resultado}")

Hipótesis 1: Con nivel de confianza del 95.0%, se rechaza la hipótesis nula
Hipótesis 2: Con nivel de confianza del 95.0%, se rechaza la hipótesis nula
Hipótesis 3: Con nivel de confianza del 95.0%, se acepta la hipótesis nula
Hipótesis 4: Con nivel de confianza del 99.0%, se rechaza la hipótesis nula
Hipótesis 5: Con nivel de confianza del 99.0%, se acepta la hipótesis nula


##### b. Calcula la media poblacional para cada variable. ¿Son correctos los resultados obtenidos por tus pruebas?

In [14]:
# Calcular la media poblacional para cada variable
media_earn = df_earnings['earn'].mean()
media_height = df_earnings['height'].mean()
media_age = df_earnings['age'].mean()

# Mostrar los resultados
print(f"Media poblacional de 'earn': {media_earn}")
print(f"Media poblacional de 'height': {media_height}")
print(f"Media poblacional de 'age': {media_age}")

Media poblacional de 'earn': 21147.296255506608
Media poblacional de 'height': 66.56883259911895
Media poblacional de 'age': 42.934471365638764


-
-
-
___0___o___
-
-
-

#### 2. Crea una función que estime la proporción de hombres en el dataset, considerando una muestra de 50 individuos, con niveles de significancia de 0,05 y 0,01. Escribe con palabras tus resultados ejecutar la función e interpreta. Compara con la proporción real.

In [15]:
def estimar_proporcion_hombres(data, n=50, alpha=0.05):
    # Tomar una muestra de tamaño n
    muestra = data['male'].sample(n=n, random_state=1)
    
    # Calcular la proporción de hombres en la muestra
    proporcion_muestra = muestra.mean()
    
    # Calcular la proporción real de hombres en la población
    proporcion_real = data['male'].mean()
    
    # Calcular el estadístico z
    p_hat = proporcion_muestra
    p = proporcion_real
    z = (p_hat - p) / np.sqrt((p * (1 - p)) / n)
    
    # Calcular el valor crítico
    z_critico = stats.norm.ppf(1 - alpha/2)
    
    # Determinar si se rechaza o no la hipótesis nula
    if abs(z) > z_critico:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se rechaza la hipótesis nula"
    else:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se acepta la hipótesis nula"
    
    return proporcion_muestra, proporcion_real, resultado

# Aplicar la función con niveles de significancia de 0.05 y 0.01
proporcion_muestra_005, proporcion_real, resultado_005 = estimar_proporcion_hombres(df_earnings, alpha=0.05)
proporcion_muestra_001, _, resultado_001 = estimar_proporcion_hombres(df_earnings, alpha=0.01)

# Mostrar los resultados
print(f"Proporción de hombres en la muestra (alpha=0.05): {proporcion_muestra_005}")
print(f"Proporción real de hombres en la población: {proporcion_real}")
print(f"Resultado (alpha=0.05): {resultado_005}")

print(f"Proporción de hombres en la muestra (alpha=0.01): {proporcion_muestra_001}")
print(f"Resultado (alpha=0.01): {resultado_001}")

Proporción de hombres en la muestra (alpha=0.05): 0.3
Proporción real de hombres en la población: 0.3716960352422907
Resultado (alpha=0.05): Con nivel de confianza del 95.0%, se acepta la hipótesis nula
Proporción de hombres en la muestra (alpha=0.01): 0.3
Resultado (alpha=0.01): Con nivel de confianza del 99.0%, se acepta la hipótesis nula


-
-
-
___0___o___
-
-
-

#### 3. Considerando el dataset como una muestra, verifica si el género (male=1 significa ‘hombre”) influye sobre el sueldo “earn” de las personas. Explica y justifica tu procedimiento.

In [21]:
# Separar los datos en dos grupos: hombres y mujeres
sueldo_hombres = df_earnings[df_earnings['male'] == 1]['earn']
sueldo_mujeres = df_earnings[df_earnings['male'] == 0]['earn']

# Realizar la prueba t para muestras independientes
t_stat, p_value = stats.ttest_ind(sueldo_hombres, sueldo_mujeres, equal_var=False)

# Nivel de significancia
alpha = 0.05

# Interpretar los resultados
if p_value < alpha:
    resultado = f"Con un nivel de significancia de {alpha}, se rechaza la hipótesis nula. Hay una diferencia significativa en los sueldos entre hombres y mujeres."
else:
    resultado = f"Con un nivel de significancia de {alpha}, no se rechaza la hipótesis nula. No hay una diferencia significativa en los sueldos entre hombres y mujeres."

# Mostrar los resultados
print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")
print(resultado)
# Explicación y justificación del procedimiento
# 
# Para verificar si el género influye sobre el sueldo, se realizó una prueba t para muestras independientes.
# Esta prueba compara las medias de dos grupos independientes (hombres y mujeres) para determinar si hay una diferencia significativa entre ellas.
# 
# Pasos realizados:
# 1. Se separaron los datos en dos grupos: hombres y mujeres.
# 2. Se realizó la prueba t para muestras independientes, asumiendo varianzas desiguales (equal_var=False).
# 3. Se calculó el estadístico t y el valor p.
# 4. Se comparó el valor p con el nivel de significancia (alpha = 0.05) para determinar si se rechaza o no la hipótesis nula.
# 
# Resultado:
# Con un nivel de significancia de 0.05, se rechaza la hipótesis nula. Esto indica que hay una diferencia significativa en los sueldos entre hombres y mujeres.

Estadístico t: 11.86459381795286
Valor p: 2.7373648004575493e-30
Con un nivel de significancia de 0.05, se rechaza la hipótesis nula. Hay una diferencia significativa en los sueldos entre hombres y mujeres.


-
-
-
___0_FiN__o___
-
-
-

### Requerimientos

1. Realiza pruebas de hipótesis, considerando el enunciado de cada una y las implementa en Python mediante funciones (5 Puntos)

2. Infiere sobre medias poblacionales y proporciones, interpretando error y significancia. (2 Puntos)

3. Plantea, interpreta e implementa hipótesis de prueba para muestras independientes. (3 Puntos)